# Dynamic Isolines

Some experiments in drawing very dynamic isolines on maps using the [Routing API](https://developer.here.com/documentation/routing/topics/resource-calculate-isoline.html) by [HERE.com](https://developer.here.com). The goal is to mimick and extend the functionality available on [mapnificent.net](http://mapnificent.net), but with data for cars and pedestrians worldwide. Mapnificent uses public transport data for selected cities.

## Playing with diameters

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html
# https://stackoverflow.com/questions/9678624/convex-hull-of-longitude-latitude-points-on-the-surface-of-a-sphere
# https://github.com/VictorDavis/GeoConvexHull/blob/master/GeoConvexHull.py

import numpy as np
from scipy.spatial import ConvexHull
points = np.random.rand(30, 2)   # 30 random points in 2-D
hull = ConvexHull(points)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(points[:,0], points[:,1], 'o')
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')

In [ ]:
>>> plt.plot(points[hull.vertices,0], points[hull.vertices,1], 'r--', lw=2)
>>> plt.plot(points[hull.vertices[0],0], points[hull.vertices[0],1], 'ro')
>>> plt.show()

In [ ]:
hull.points

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path

# Fixing random state for reproducibility
np.random.seed(19680801)

fig, ax = plt.subplots()

for i in range(12):
    vertices = (np.random.random((2, 2)) - 0.5) * 6.0
    # path = Path(vertices)
    ax.plot(vertices[:, 0], vertices[:, 1], color='b')

plt.show()

In [ ]:
(np.random.random((5, 2)))

In [ ]:
import itertools
list(itertools.combinations([1, 2, 3, 4, 5], 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import Bbox
from matplotlib.path import Path

# Fixing random state for reproducibility
np.random.seed(19680801)

left, bottom, width, height = (-1, -1, 2, 2)
rect = plt.Rectangle((left, bottom), width, height, facecolor="#aaaaaa")

fig, ax = plt.subplots()
ax.add_patch(rect)

bbox = Bbox.from_bounds(left, bottom, width, height)

for i in range(12):
    vertices = (np.random.random((2, 2)) - 0.5) * 6.0
    path = Path(vertices)
    if path.intersects_bbox(bbox):
        color = 'r'
    else:
        color = 'b'
    ax.plot(vertices[:, 0], vertices[:, 1], color=color)

plt.show()

## Real stuff

In [ ]:
# ! conda install -y -c anaconda shapely 

In [1]:
import os
import time
from math import log, fabs
from functools import partial

import requests
from ipywidgets import IntSlider, VBox, HBox, Dropdown, Button, HTML, Layout
from ipyleaflet import Marker, Polyline
from geographiclib.geodesic import Geodesic
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
geod = Geodesic.WGS84

def calc_perim_area(path):
    "Calculate and return geodesic perimeter and area of some path."
    p = geod.Polygon()
    for lat, lon in path:
        p.AddPoint(lat, lon)
    num, perim, area = p.Compute()
    return num, perim, area

In [3]:
def zoom_for_bbox(lon_min: float,
                  lat_min: float,
                  lon_max: float,
                  lat_max: float) -> int:
    """
    Return zoom level for some bounding box.
    """
    lat_diff = fabs(lat_max - lat_min)
    lon_diff = fabs(lon_max - lon_min)
    max_diff = max(lon_diff, lat_diff)

    if max_diff < 360 / 2**20:
        zoom_level = 21
    else:
        zoom_level = int(-1 * ((log(max_diff) / log(2)) - (log(360) / log(2))))
        if zoom_level < 1:
            zoom_level = 1
    return zoom_level

In [4]:
# maybe use this for toggeling rangetype and mode
# (but would need newer font-awesome version in ipywidgets)

def do_toggle(sender):
    # print(sender)
    # sender.description = '111' if sender.description == '000' else '000'
    sender.icon = 'trash' if sender.icon == 'ban' else 'ban'

tgl = Button(icon='trash', layout=Layout(width='30px'))
tgl.on_click(do_toggle)
tgl

Button(icon='trash', layout=Layout(width='30px'), style=ButtonStyle())

In [53]:
class DynamicIsolines(object):
    """
    A class to draw isolines dynamically around some center on a map.

    The markers at the center have a popup with some UI to see and change
    the isoline type (time/distance) range (secods/meters), resolution
    (meters) and mode (car/pedestrian). Modifying these will update the
    isoline on the map.

    Isolines are cached for (lat, lon, range) keys and shown with red
    outlines when newly fetched via the API, or green ones when reused
    from previously cached API calls.
    
    Done:

    - add delete button
    - add slider for range
    - add slider for resolution
    - start with empty map at zoom 1
    - add initial isolines to map
    - use rangetype (time/distance)
    - use mode (car/pedestrian)
    - make isoline border dashed if center lies outside

    TODO:

    - use fontawesome icons car-side/walking icons for mode
    - use fontawesome icons ruler/clock icons for rangetype
    - add auto-resize toggle icon for isoline *class*
    - add HERE appcode/appid to some config for `__init__`
    - add key modifier like Shift to `self.map_click`
    - add diameter (The diameter of a polygon is the largest distance between 
      any pair of vertices. In other words, it is the length of the longest
      polygon diagonal (e.g., straight line segment joining two vertices).
      https://unicode-table.com/en/2300/  - ⌀
    
    NOTES:
    
    - sliders don't have a name attribute, but polylines and markers do
    - callbacks registered with marker.on_move() don't get a marker/owner argument 
    - callbacks registered with map.on_interaction() don't get modifier keys pressed on the keyboard
    - marker popups don't show/work nicely in Jupyter Lab (with or without Sidecar)
    """
    
    default_iso = dict(
        range=300,
        rangetype='time',
        resolution=50,
        mode='car',
        from_cache=False
    )
    
    stats_format = 'L:&nbsp;{:.0f}&nbsp;pts, P:&nbsp;{:.0f}&nbsp;m, A:&nbsp;{:.0f}&nbsp;m<sup>2</sup>'
    
    def __init__(self, a_map, isolines=None):
        self.m = a_map
        self.m.on_interaction(self.map_click)

        self.isolines = isolines or []
        self.cache = {}
        self.auto_resize = False

        self.app_id = os.getenv('HEREMAPS_APP_ID', 'rextLVOZWwI3G7bulbb3')
        self.app_code = os.getenv('HEREMAPS_APP_CODE', 'CexmP6l-DmHVYw0e8VXnhQ')

        for i, iso0 in enumerate(self.isolines):
            iso = self.default_iso.copy()
            iso.update(iso0)
            self.isolines[i] = iso
            self.make_iso_interactive(iso)            
            obj = iso.get('obj', self.get_isoline_cached(iso))
            iso['obj'] = obj
            self.render_on_map(iso)

        self.resize()

    # callbacks

    def rangetype_changed(self, change, iso):
        "Callback for events when a `rangetype` dropdown was changed."
        if change['type'] == 'change' and change['name'] == 'value':
            self.clean(iso, incl_slider=False)
            iso['rangetype'] = change['new'].lower()
            desc = 'Time (s)' if iso['rangetype']=='time' else 'Dist. (m)'
            iso['range_slider'].description = desc
            iso['obj'] = self.get_isoline_cached(iso)
            self.render_on_map(iso, add_marker=False)
            if self.auto_resize:
                self.resize()

    def mode_changed(self, change, iso):
        "Callback for events when a `mode` dropdown was changed."
        if change['type'] == 'change' and change['name'] == 'value':
            self.clean(iso, incl_slider=False)
            iso['mode'] = change['new'].lower()
            iso['obj'] = self.get_isoline_cached(iso)
            self.render_on_map(iso, add_marker=False)
            num, perim, area = map(fabs, calc_perim_area(iso['path']))
            iso['stats'].value = self.stats_format.format(num, perim, area)
            if self.auto_resize:
                self.resize()

    def obs_range_slider(self, change, iso):
        "Callback for events when a range slider was moved."
        self.clean(iso, incl_slider=False)
        iso['range'] = change.owner.value
        iso['obj'] = self.get_isoline_cached(iso)
        self.render_on_map(iso, add_marker=False)
        if self.auto_resize:
            self.resize()

    def obs_reso_slider(self, change, iso):
        "Callback for events when a resolution slider was moved."
        self.clean(iso, incl_slider=False)
        iso['resolution'] = change.owner.value
        iso['obj'] = self.get_isoline_cached(iso)
        self.render_on_map(iso, add_marker=False)
        if self.auto_resize:
            self.resize()

    def move_marker(self, event, location, iso) -> None:
        "Callback for events when marker was moved."
        self.clean(iso)
        iso['loc'] = tuple(location)
        iso['obj'] = self.get_isoline_cached(iso)
        self.render_on_map(iso)
        if self.auto_resize:
            self.resize()
    
    def click_delete(self, event, iso) -> None:
        "Callback for events when button was clicked."
        self.clean(iso)
        self.isolines.remove(iso)

    def add_isoline(self, **kwargs):
        "Callback for events when map was clicked."
        iso = self.default_iso.copy()
        iso['loc'] = kwargs['loc']
        self.make_iso_interactive(iso)            
        obj = iso.get('obj', self.get_isoline_cached(iso))
        iso['obj'] = obj
        self.isolines.append(iso)
        self.render_on_map(iso)
        if self.auto_resize:
            self.resize()

    def map_click(self, **kwargs):
        "Callback for events when map was clicked."
        if kwargs.get('type') == 'click':
            kwargs['loc'] = tuple(kwargs.get('coordinates'))
            del kwargs['coordinates']
            self.add_isoline(**kwargs)
            
    # normal methods
    
    def clean(self, iso, incl_slider=True):
        "Remove current marker and/or isoline from map."
        if incl_slider:
            self.m -= iso['marker']
        self.m -= iso['polyline']
    
    def make_iso_interactive(self, iso):
        "Make an iso object respond to UI."
        range_slider = IntSlider(
            value=iso['range'],
            min=60, max=600, step=10,
            description='Time (s)' if iso['rangetype']=='time' else 'Dist. (m)')
        obs_range_slider = partial(self.obs_range_slider, iso=iso)
        range_slider.observe(obs_range_slider, names='value')
        iso['range_slider'] = range_slider

        reso_slider = IntSlider(
            value=iso['resolution'],
            min=10, max=200, step=10,
            description='Resol. (m)')
        obs_reso_slider = partial(self.obs_reso_slider, iso=iso)
        reso_slider.observe(obs_reso_slider, names='value')
        iso['reso_slider'] = reso_slider

        iso['stats'] = HTML('')

    def get_isoline_cached(self, iso):
        "Get isoline data from cache or by executing an API call if needed."
        fields = 'loc range rangetype resolution mode'.split()
        key = tuple(iso[f] for f in fields)
        if key not in self.cache:
            obj = self.get_isoline(iso)
            self.cache[key] = obj
            iso['from_cache'] = False
        else:
            obj = self.cache[key]
            iso['from_cache'] = True
        iso['obj'] = obj
        return obj

    def get_isoline(self, iso):
        "Execute API call to get path data for isoline object."
        url = 'https://isoline.route.api.here.com' \
              '/routing/7.2/calculateisoline.json'
        params = dict(
            app_id=self.app_id, 
            app_code=self.app_code,
            start='geo!{lat},{lon}'.format(lat=iso['loc'][0], lon=iso['loc'][1]),
            mode='fastest;{mode};traffic:disabled'.format(mode=iso['mode'].lower()),
            rangetype=iso['rangetype'], # time/distance
            range=str(iso['range']),  # seconds/meters
            resolution=str(iso['resolution']),  # meters
            #departure='now', # 2018-07-04T17:00:00+02
        )
        t0 = time.time()
        return requests.get(url, params=params).json()

    def make_marker(self, iso):
        "Return a merker with popup to be put on a map."
        del_btn = Button(icon='trash', layout=Layout(width='30px'))
        click_delete = partial(self.click_delete, iso=iso)
        del_btn.on_click(click_delete)
        
        rangetype_dd = Dropdown(
            options=['Time', 'Distance'],
            value=iso['rangetype'].capitalize(),
            description='',
            disabled=False,
            layout=Layout(width='180px')
        )
        rangetype_changed = partial(self.rangetype_changed, iso=iso)
        rangetype_dd.observe(rangetype_changed)

        mode_dd = Dropdown(
            options=['Car', 'Pedestrian'],
            value=iso['mode'].capitalize(),
            description='',
            disabled=False,
            layout=Layout(width='180px')
        )
        mode_changed = partial(self.mode_changed, iso=iso)
        mode_dd.observe(mode_changed)
        
        here_url = 'https://developer.here.com'
        isoline_url = 'https://developer.here.com/documentation/routing/topics/resource-calculate-isoline.html'
        popup = VBox([
            HBox([HTML('<b>Isoline</b>'), 
                  rangetype_dd,
                  mode_dd, 
                  del_btn
            ]), 
            iso['range_slider'], 
            iso['reso_slider'],
            iso['stats'],
            HTML(f'Using the <a href="{isoline_url}">Routing&nbsp;API</a> by <a href="{here_url}">HERE.com</a>.')
        ])
        # popup = iso['range_slider']
        lat, lon = iso['loc']
        marker = Marker(location=(lat, lon), popup=popup, name='isoline_marker')
        iso['marker'] = marker
        move_marker = partial(self.move_marker, iso=iso)
        marker.on_move(move_marker)
        return marker
        
    def render_on_map(self, iso, add_marker=True):
        "Render isoline object on map."
        obj = iso['obj']
        center = obj['response']['center']
        lat, lon = center['latitude'], center['longitude']
        if add_marker:
            self.m += self.make_marker(iso)
        for isoline in obj['response']['isoline']:
            shape = isoline['component'][0]['shape']
            path = [tuple(map(float, pos.split(','))) for pos in shape]
            color = 'green' if iso['from_cache'] else 'red'
            dash_array = '5,5' if not Polygon(path).contains(Point(iso['loc'])) else None
            polyline = Polyline(locations=path, color=color, weight=2, fill=True, dash_array=dash_array, name='isoline')
            iso['polyline'] = polyline
            iso['path'] = path
            num, perim, area = map(fabs, calc_perim_area(path))
            iso['stats'].value = self.stats_format.format(num, perim, area)
            self.m += polyline

    def resize(self):
        "Resize map, resetting map center and zoom level to show all isolines."
        # reset center
        locs = [iso['loc'] for iso in self.isolines]
        len_locs = len(locs)
        if len_locs == 0:
            return
        self.m.center = sum(loc[0] for loc in locs) / len_locs, sum(loc[1] for loc in locs) / len_locs

        # reset zoom level
        mins, maxs = [], []
        for iso in self.isolines:
            path = iso.get('path', None)
            if path:
                mins.append(min(path))
                maxs.append(max(path))
        self.m.zoom = zoom_for_bbox(*min(mins), *max(maxs))

In [47]:
# test for https://github.com/jupyter-widgets/ipyleaflet/issues/259
from ipyleaflet import Map, Polyline
Map(center=(52.5, 13.4), zoom=11)
_ += Polyline(locations=[(52.45, 13.35), (52.55, 13.45), (52.55, 13.35), (52.45, 13.35)], fill=True)
_

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

## Create a map

In [16]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

In [17]:
APP_ID = os.getenv('HEREMAPS_APP_ID', 'rextLVOZWwI3G7bulbb3')
APP_CODE = os.getenv('HEREMAPS_APP_CODE', 'CexmP6l-DmHVYw0e8VXnhQ')

In [23]:
attribution = '<a href="http://here.com">HERE</a>'
url_pat = ("https://1.{maptype}.maps.api.here.com"
           "/maptile/2.1/{tiletype}/newest/{scheme}/{{z}}/{{x}}/{{y}}/{tilesize}/{format}"
           f"?app_id={APP_ID}&app_code={APP_CODE}")
params = dict(
    maptype  = 'traffic',
    tiletype = 'traffictile',
    scheme   = 'normal.day',
    tilesize = '256',
    format   = 'png8',
    app_id   = APP_ID,
    app_code = APP_CODE
)
basemap = dict(url=url_pat.format(**params), attribution=attribution)

## Add Isolines

**Warning:** The marker popups will show best inside classic Jupyter notebooks. Experiments with Jupyter Lab showed a couple of issues, quite likely related to styling: some elements are not shown when using Jupyter Lab's Dark theme, sliders don't seem to work as expected.

In [54]:
m = Map(center=(52.5, 13.4), zoom=13, basemap=basemap)

# from sidecar import Sidecar
iso0 = dict(loc=(52.5, 13.4), range=300, rangetype='time', resolution=20, mode='car')
# iso1 = dict(loc=(52.51, 13.41), range=600, rangetype='time', resolution=100, mode='car')
dyniso = DynamicIsolines(m, isolines=[iso0])
# with Sidecar(title='Isolines'):
#     display(dyniso.m)
dyniso.m

Map(basemap={'url': 'https://1.traffic.maps.api.here.com/maptile/2.1/traffictile/newest/normal.day/{z}/{x}/{y}…

In [ ]:
dyniso.add_isoline(loc=(52.52, 13.42), range=300, rangetype='time', resolution=100)

In [ ]:
dyniso.resize()

In [ ]:
dyniso.auto_resize = True